# Setting things up

## About this notebook

In this notebook, we embark on a journey to enhance the predictive power of the Titanic Dataset through sophisticated preprocessing using the ``BinaryCarver`` pipeline. Designed to maximize associations in the data, ``BinaryCarver`` is a robust Python tool capable of discretizing any type of data—whether it be quantitative or qualitative. Our specific focus is on preparing the dataset for binary classification tasks, such as predicting survival outcomes.

The Titanic Dataset, derived from the iconic 1912 Titanic passenger information, provides a diverse set of features ranging from socio-economic status and age to cabin location. Leveraging ``BinaryCarver``, we aim to perform association-maximizing discretization, refining both quantitative and qualitative features to create a finely tuned dataset for our binary classification endeavors.

Throughout this notebook, we'll delve into the intricacies of ``BinaryCarver``'s discretization pipeline, exploring its capabilities in handling a variety of data types. Whether it's transforming passenger ages or classifying fares, ``BinaryCarver``'s adaptability ensures that every feature is optimally represented for our classification tasks.

Join us in this exploration as we harness the power of ``BinaryCarver`` to preprocess the Titanic Dataset. Through effective feature engineering and discretization, we strive to create a dataset that not only captures the nuances of the Titanic passenger profiles but also sets the stage for the development of accurate and impactful binary classification models.

Let's dive in and uncover the potential of ``BinaryCarver`` in transforming the Titanic Dataset for optimal predictive modeling.


## Installation

In [1]:
# %pip install AutoCarver[jupyter]

## Titanic Data

In this example notebook, we will use the Titanic dataset.

The Titanic dataset is a well-known and frequently used dataset in the field of machine learning and data science. It provides information about the passengers on board the Titanic, the famous ship that sank on its maiden voyage in 1912. The dataset is often used for predictive modeling, classification, and regression tasks.

The dataset includes various features such as passengers' names, ages, genders, ticket classes, cabin information, and whether they survived or not. The primary goal when working with the Titanic dataset is often to build predictive models that can infer whether a passenger survived or perished based on their individual characteristics (binary classification).

In [1]:
import pandas as pd

# URL to the Titanic dataset on Kaggle
titanic_url = "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"

# Use pandas to read the CSV file directly from the URL
titanic_data = pd.read_csv(titanic_url)

# Display the first few rows of the dataset
titanic_data.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


## Target type and Carver selection

In [2]:
target = "Survived"

titanic_data[target].value_counts(dropna=False)

Survived
0    545
1    342
Name: count, dtype: int64

The target ``"Survived"`` is a binary target of type ``int64`` used in a classification task. Hence we will use ``AutoCarver.BinaryCarver`` and ``AutoCarver.selectors.ClassificationSelector`` in following code blocks.

## Data Sampling

In [3]:
from sklearn.model_selection import train_test_split

# stratified sampling by target
train_set, dev_set = train_test_split(titanic_data, test_size=0.33, random_state=42, stratify=titanic_data[target])

# checking target rate per dataset
train_set[target].mean(), dev_set[target].mean()

(np.float64(0.38552188552188554), np.float64(0.3856655290102389))

## Setting up Features to Carver

In [4]:
train_set.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
617,0,3,Mr. Antoni Yasbeck,male,27.0,1,0,14.4542
489,0,1,Mr. Harry Markland Molson,male,55.0,0,0,30.5000
871,1,3,Miss. Adele Kiamie Najib,female,15.0,0,0,7.2250
654,0,3,Mrs. John (Catherine) Bourke,female,32.0,1,1,15.5000
653,0,3,Mr. Alexander Radeff,male,27.0,0,0,7.8958


In [5]:
# column data types
train_set.dtypes

Survived                     int64
Pclass                       int64
Name                        object
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard      int64
Fare                       float64
dtype: object

In [6]:
# values taken by Parents/Children Aboard
train_set["Parents/Children Aboard"].value_counts()

Parents/Children Aboard
0    438
1     87
2     60
3      3
5      3
4      2
6      1
Name: count, dtype: int64

In [7]:
# values taken by Pclass
train_set["Pclass"].value_counts()

Pclass
3    326
1    142
2    126
Name: count, dtype: int64

The feature ``"Pclass"`` is of type ``"int64"``, but it can be considered a qualitative ordinal feature rather than a quantitative discrete feature (socio-economic status). Thus we will add it to the list of ``ordinal_features`` and set the ordering of its values in ``values_orders`` (string values). 

``"Sex"`` is the only quantitative categorical feature, it's added to the list of ``qualitative_features``.

``"Fare"`` is the only quantitative continuous features, whilst ``"Age"``, ``"Siblings/Spouses Aboard"`` and ``"Parents/Children Aboard"`` can be considered as quantitative discrete features. Those four features will be added to the list of ``quantitative_features``.

In [8]:
from AutoCarver import Features

# initiating Features to carve
features = Features(
    categoricals=["Sex"],
    quantitatives=["Age", "Fare", "Siblings/Spouses Aboard", "Parents/Children Aboard"],
    ordinals={"Pclass": ["1", "2", "3"]},  # user-specified ordering for ordinal features
)
features["Pclass"], features["Sex"], features["Age"]

C:\Users\defra\Desktop\git\PROJECTS\AutoCarver\AutoCarver\combinations\utils\combination_evaluator.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


(Ordinal('Pclass'), Categorical('Sex'), Quantitative('Age'))

# Using AutoCarver

## AutoCarver settings

### Representativness of modalities

The attribute ``min_freq`` allows one to choose the minimum frequency per basic modalities. It is used:

- For quantitative features, to define the number of quantiles to initialy discretize the features with.

- For qualitative features, to define the threshold under which a modality is grouped to either a default value or its closest modality.

In [9]:
min_freq = 0.05

**Tip:** should be set between ``0.01`` (slower, preciser, less robust) and ``0.2`` (faster, more robust)

### Desired number of modalities

The attribute ``max_n_mod`` allows one to choose the maximum number of modalities per carved feature. It is used by **Carvers** has the upper limit of number of modalities per consecutive combination of modalities.

In [10]:
max_n_mod = 5

**Tip:** should be set between ``3`` (faster, more robust) and ``7`` (slower, preciser, less robust)

### Association metric

The attribute ``sort_by`` allows one to choose the association metric used to sort combinations. Combinations of grouped modalities are ranked according to the specified modalities and the best ranked viable combination is returned by **Carvers**.

In [11]:
# For BinaryCarver, to be choosen amongst ["tschuprowt", "cramerv"]
# sort_by = "tschuprowt"  # "cramerv"

**Tip:** use ``"tschuprowt"`` for more robust, or less output modalities, use ``"cramerv"`` for more output modalities.

### Grouping NaNs

The attribute ``dropna`` allows one to choose whether or not ``numpy.nan`` should be grouped with another modality. If set to ``True``, **Carvers** will first find the most suitable combination of non-NaN values, and then test out all possible combinations with ``numpy.nan``.

In [12]:
dropna = False  # anyway, there are no numpy.nan in this dataset

### Optional attributes

#### Minimal frequency per carved modality

The attribute ``min_freq_mod`` allows one to choose the minimum frequency per output modality. It is used by **Carvers** in viability tests to put aside combinations that are not frequent enough in train or dev sets. By default, it is set to ``min_freq/2``.

In [13]:
# min_freq_mod = None  # for 0.05,  at least 5 % of observations per output modality in train and dev sets 

#### Type of output carved features

The attribute ``output_dtype`` allows one to choose the output type:

* Use ``"float"`` for integer output (default)
* Use ``"str"`` for string output

## TODO


In [15]:
ordinal_encoding = True


## Fitting AutoCarver

* First, all qualitative features are discretized:
    1. Using ``StringDiscretizer`` to convert them to ``str`` if not already the case
    2. For qualitative ordinal features: using ``OrdinalDiscretizer`` for under-represented values (less frequent than ``min_freq=0.05``) to be grouped with its closest modality
    3. For qualitative categorical features: using ``CategoricalDiscretizer`` for under-represented values (less frequent than ``min_freq=0.05``) to be grouped with a default value (``str_default="__OTHER__"``)

* Second, all quantitative features are discretized:
    1. Using ``ContinuousDiscretizer`` for quantile discretization that keeps track of over-represented values (more frequent than ``min_freq=0.05``)
    2. Using ``OrdinalDiscretizer`` for any remaining under-represented values (less frequent than ``min_freq/2=0.025``) to be grouped with its closest modality

* Third, all features are carved following this recipe, for all classes of ``train_set[target]`` (except one):
    1. The raw distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the discretization step
    2. Grouping modalities: all consecutive combinations of modalities are applied to ``train_set``
    3. Computing associations: the association metric (``sort_by="cramerv"``) is computed with the provided target ``train_set[target]``
    4. Combinations are sorted in descending order by association value
    5. Testing robustness: finds the first combination that checks the following:
        - Representativness of modalities on ``train_set`` and ``dev_set`` (all should be more frequent than ``min_freq_mod``)
        - Distinct target rates per consecutive modalities on ``train_set`` and ``dev_set`` 
        - No inversion of target rates between ``train_set`` and ``dev_set`` (same ordering of modalities by target rate)
    6. (Optional) If requested via ``dropna=True``, and if any, all combinations of modalities with ``numpy.nan`` are applied to ``train_set`` and steps 3. and 4. are run
    7. The carved distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the carving step

In [ ]:
from AutoCarver import BinaryCarver

# intiating AutoCarver
auto_carver = BinaryCarver(
    features=features,
    min_freq=min_freq,
    ordinal_encoding=ordinal_encoding,
    verbose=True,  # showing statistics
    copy=True,  # whether or not to return a copy of the input dataset
)

# fitting on training sample, a dev sample can be specified to evaluate carving robustness
train_set_processed = auto_carver.fit_transform(train_set, train_set[target], X_dev=dev_set, y_dev=dev_set[target])

------
--- [QuantitativeDiscretizer] Fit Features(['Age', 'Fare', 'Siblings/Spouses Aboard', 'Parents/Children Aboard'])
 - [ContinuousDiscretizer] Fit Features(['Age', 'Fare', 'Siblings/Spouses Aboard', 'Parents/Children Aboard'])
 - [OrdinalDiscretizer] Fit Features(['Age', 'Fare', 'Parents/Children Aboard'])
------

------
--- [QualitativeDiscretizer] Fit Features(['Sex', 'Pclass'])
 - [StringDiscretizer] Fit Features(['Pclass'])
 - [OrdinalDiscretizer] Fit Features(['Pclass'])
 - [CategoricalDiscretizer] Fit Features(['Sex'])
------

---------
------ [BinaryCarver] Fit Features(['Sex', 'Pclass', 'Age', 'Fare', 'Siblings/Spouses Aboard', 'Parents/Children Aboard'])
--- [BinaryCarver] Fit Categorical('Sex') (1/6)
 [BinaryCarver] Raw distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 male 
 0.1878 
 0.6364 
 
 
 female 
 0.7315 
 0.3636 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.1949 
 0.6655 
 
 
 0.7653 
 0.3345

Testing robustness    :   0%|          | 0/1 [00:00<?, ?it/s]



 [BinaryCarver] Carved distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 male 
 0.1878 
 0.6364 
 
 
 female 
 0.7315 
 0.3636 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.1949 
 0.6655 
 
 
 0.7653 
 0.3345

...

--- [BinaryCarver] Fit Quantitative('Parents/Children Aboard') (6/6)
 [BinaryCarver] Raw distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 0.0e+00 
 0.3447 
 0.7374 
 
 
 0.0e+00 < x <= 1.0e+00 
 0.5057 
 0.1465 
 
 
 1.0e+00 < x <= 2.0e+00 
 0.5167 
 0.1010 
 
 
 2.0e+00 < x 
 0.3333 
 0.0152 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.3475 
 0.8055 
 
 
 0.6774 
 0.1058 
 
 
 0.4500 
 0.0683 
 
 
 0.1667 
 0.0205

Testing robustness    :   0%|          | 0/7 [00:00<?, ?it/s]



 [BinaryCarver] Carved distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 0.0e+00 
 0.3447 
 0.7374 
 
 
 0.0e+00 < x 
 0.5000 
 0.2626 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.3475 
 0.8055 
 
 
 0.5439 
 0.1945

## AutoCarver analysis

### Carving Summary

In [17]:
features.summary

content  \
feature                                 cramerv  tschuprowt n_mod label                                            
Categorical('Sex')                      0.533719 0.533719   2     male                                      male   
                                                                  female                                  female   
Ordinal('Pclass')                       0.300144 0.300144   2     1 to 2                                  [2, 1]   
                                                                  3                                            3   
Quantitative('Age')                     0.139166 0.139166   2     x <= 8.0e+00                      x <= 8.0e+00   
                                                                  8.0e+00 < x                        8.0e+00 < x   
Quantitative('Fare')                    0.295325 0.295325   2     x <= 5.2e+01                      x <= 5.2e+01   
                                                                  5.2e+01 < x                        5.2e+01 < x   
Quantitative('Siblings/Spouses Aboard') 0.139722 0.117492   3     x <= 0.0e+00                      x <= 0.0e+00   
                                                                  0.0e+00 < x <= 1.0e+00  0.0e+00 < x <= 1.0e+00   
                                                                  1.0e+00 < x                        1.0e+00 < x   
Quantitative('Parents/Children Aboard') 0.136439 0.136439   2     x <= 0.0e+00                      x <= 0.0e+00   
                                                                  0.0e+00 < x                        0.0e+00 < x   

                                                                                          target_rate  \
feature                                 cramerv  tschuprowt n_mod label                                 
Categorical('Sex')                      0.533719 0.533719   2     male                       0.187831   
                                                                  female                     0.731481   
Ordinal('Pclass')                       0.300144 0.300144   2     1 to 2                     0.548507   
                                                                  3                          0.251534   
Quantitative('Age')                     0.139166 0.139166   2     x <= 8.0e+00               0.636364   
                                                                  8.0e+00 < x                0.365455   
Quantitative('Fare')                    0.295325 0.295325   2     x <= 5.2e+01               0.319838   
                                                                  5.2e+01 < x                0.710000   
Quantitative('Siblings/Spouses Aboard') 0.139722 0.117492   3     x <= 0.0e+00               0.361386   
                                                                  0.0e+00 < x <= 1.0e+00     0.500000   
                                                                  1.0e+00 < x                0.269231   
Quantitative('Parents/Children Aboard') 0.136439 0.136439   2     x <= 0.0e+00               0.344749   
                                                                  0.0e+00 < x                0.500000   

                                                                                          frequency  
feature                                 cramerv  tschuprowt n_mod label                              
Categorical('Sex')                      0.533719 0.533719   2     male                     0.636364  
                                                                  female                   0.363636  
Ordinal('Pclass')                       0.300144 0.300144   2     1 to 2                   0.451178  
                                                                  3                        0.548822  
Quantitative('Age')                     0.139166 0.139166   2     x <= 8.0e+00             0.074074  
                                                                  8.0e+00 

* As requested with ``output_dtype="float"``, output labels are integers of ranks of modalities

* For quantitative feature ``Age``, the selected combination of modalities groups ages as follows:
    * modality ``0``: lower or equal to 4 years old (``content==["x <= 4.000e+00"]``)
    * modality ``1``: ages higher than 4 years old (``content==["4.000e+00 < x "]``)

* For qualitative categorical feature ``Sex``, the selected combination of modalities has left modalities ``content=["male"]`` in modality ``0`` and ``content=["female"]`` in modality ``1`` (no combination possible)

* For qualitative ordinal feature ``Pclass``, the selected combination of modalities socio-economic status as follows:
    * modality ``0``: upper and middle classes (``content==[1, 2]``) 
    * modality ``1``: lower class (``content==[3]``). 
    * The user-provided ordering of modalities has been preserved.

### Detailed overview of tested combinations

In [18]:
features["Pclass"].history

,info,cramerv,tschuprowt,combination,n_mod,dropna,train,viable,dev
0,Raw distribution,0.321044,0.269965,"{'1': '1', '2': '2', '3': '3'}",3,False,NaN,NaN,NaN
1,Best for tschuprowt and max_n_mod=5,0.300144,0.300144,"{'1': '1', '2': '1', '3': '3'}",2,False,"{'viable': True, 'info': ''}",True,"{'viable': True, 'info': ''}"
2,Not checked,0.321044,0.269965,"{'1': '1', '2': '2', '3': '3'}",3,False,NaN,NaN,NaN
3,Not checked,0.265643,0.265643,"{'1': '1', '2': '2', '3': '2'}",2,False,NaN,NaN,NaN


* The most associated combination (the first tested out, where ``viability_message!=["Raw X distribution"]``) groups ``Pclass==1`` with ``Pclass==2`` and leaves ``Pclass==3`` as its own modality

* For feature feature ``Pclass``, the 1st combination is passes the tests:
    - ``viability_message!=["Combination robust between X and X_dev"]``
    - Tschuprow's T with ``Survived`` is ``0.300144`` for this combination
    - Following combinations (less associated with the target) where not tested: ``viability_message==["Not checked"]``

* For all combinations ``grouping_nan==False`` means that it is not a combination in which NaNs are being groupedwith other modalities (as requested with ``dropna=False``)

## Saving and Loading AutoCarver
### Saving

All **Carvers** can safely be stored as a .json file.

In [19]:
auto_carver.save("binary_carver.json")

### Loading

**Carvers** can safely be loaded from a .json file.

In [20]:
auto_carver = BinaryCarver.load("binary_carver.json")

## Applying AutoCarver

In [21]:
dev_set_processed = auto_carver.transform(dev_set)

In [22]:
dev_set_processed[auto_carver.features].apply(lambda u: u.value_counts(dropna=False, normalize=True))

,Sex,Pclass,Age,Fare,Siblings/Spouses Aboard,Parents/Children Aboard
0.0e+00 < x,NaN,NaN,NaN,NaN,NaN,0.194539
0.0e+00 < x <= 1.0e+00,NaN,NaN,NaN,NaN,0.242321,NaN
1 to 2,NaN,0.450512,NaN,NaN,NaN,NaN
1.0e+00 < x,NaN,NaN,NaN,NaN,0.075085,NaN
3,NaN,0.549488,NaN,NaN,NaN,NaN
5.2e+01 < x,NaN,NaN,NaN,0.167235,NaN,NaN
8.0e+00 < x,NaN,NaN,0.935154,NaN,NaN,NaN
female,0.334471,NaN,NaN,NaN,NaN,NaN
male,0.665529,NaN,NaN,NaN,NaN,NaN
x <= 0.0e+00,NaN,NaN,NaN,NaN,0.682594,0.805461


In [23]:
dev_set_processed[auto_carver.features].apply(lambda u: u.value_counts(dropna=False, normalize=True))

,Sex,Pclass,Age,Fare,Siblings/Spouses Aboard,Parents/Children Aboard
0.0e+00 < x,NaN,NaN,NaN,NaN,NaN,0.194539
0.0e+00 < x <= 1.0e+00,NaN,NaN,NaN,NaN,0.242321,NaN
1 to 2,NaN,0.450512,NaN,NaN,NaN,NaN
1.0e+00 < x,NaN,NaN,NaN,NaN,0.075085,NaN
3,NaN,0.549488,NaN,NaN,NaN,NaN
5.2e+01 < x,NaN,NaN,NaN,0.167235,NaN,NaN
8.0e+00 < x,NaN,NaN,0.935154,NaN,NaN,NaN
female,0.334471,NaN,NaN,NaN,NaN,NaN
male,0.665529,NaN,NaN,NaN,NaN,NaN
x <= 0.0e+00,NaN,NaN,NaN,NaN,0.682594,0.805461


# Feature Selection
## Selectors settings

### Features to select from

Here all features have been carved using ``BinaryCarver``, hence all features are qualitative.


### Number of features to select

The attribute ``n_best_per_type`` allows one to choose the number of features to be selected per data type (quantitative and qualitative).

In [24]:
n_best_per_type = 2  # here the number of features is low, ClassificationSelector will only be used to compute useful statistics

## Using Selectors

In [25]:
from AutoCarver.selectors import CramervMeasure, TschuprowtMeasure
from AutoCarver.selectors import CramervFilter, TschuprowtFilter
measures = [CramervMeasure(), TschuprowtMeasure()]
filters = [CramervFilter(), TschuprowtFilter()]

In [26]:
from AutoCarver import ClassificationSelector

# select the most target associated qualitative features
feature_selector = ClassificationSelector(
    features=features,
    n_best_per_type=4,
    verbose=True,  # displays statistics
)
best_features = feature_selector.select(train_set_processed, train_set_processed[target])

 [ClassificationSelector] Selected Features


,feature,NaNMeasure,ModeMeasure,TschuprowtMeasure,Tschuprowt_Rank,TschuprowtFilter,TschuprowtFilter_With
0,Categorical('Sex'),0.0000,0.6364,0.5337,6,0.0000,itself
1,Ordinal('Pclass'),0.0000,0.5488,0.3001,5,0.0988,Sex
3,Quantitative('Fare'),0.0000,0.8316,0.2953,4,0.3922,Pclass
2,Quantitative('Age'),0.0000,0.9259,0.1392,3,0.1002,Sex
5,Quantitative('Parents/Children Aboard'),0.0000,0.7374,0.1364,2,0.4666,Age
4,Quantitative('Siblings/Spouses Aboard'),0.0000,0.6801,0.1175,1,0.4060,Parents/Children Aboard


In [27]:
best_features

Features(['Sex', 'Pclass', 'Fare', 'Age'])

In [34]:
import pandas as pd
from AutoCarver.features import BaseFeature

def format_measure(feature: BaseFeature, measure: dict) -> dict:
    """ Format measure for display """
    # formatting measure
    formatted = {}
    for measure_name, measure_value in measure.items():
        # adding feature name and measure value
        formatted.update({"feature": feature, measure_name: measure_value.get("value")})
        # adding correlation with
        if not measure_value.get("info", {}).get("is_default"):
            correlation_with = measure_value.get("info", {}).get("correlation_with")
            # formatting target correlation
            if correlation_with != "target" and correlation_with is not None:
                formatted.update({f"{measure_name}_with": correlation_with})

    return formatted

def format_ranked_features(features: list[BaseFeature]) -> pd.DataFrame:
    """  formats ranked features for display """

    # adding measures and filters
    measures = []
    for feature in features:
        measures.append(format_measure(feature, {**feature.measures, **feature.filters}))

    # finding a sorting measure
    ranks = [col for col in measures[0].keys() if col.endswith("_Rank")]
    if len(ranks) > 0:
        sort_by = ranks[0]
        return pd.DataFrame(measures).sort_values(by=sort_by, ascending=False)
    return pd.DataFrame(measures)


In [37]:
features[1].filters

{'CramervFilter': {'value': 0.3922454314610025,
  'threshold': 1.0,
  'valid': True,
  'info': {'higher_is_better': False, 'correlation_with': 'Fare'}},
 'TschuprowtFilter': {'value': 0.3922454314610025,
  'threshold': 1.0,
  'valid': True,
  'info': {'higher_is_better': False, 'correlation_with': 'Fare'}}}

In [36]:
format_ranked_features(features)

,feature,NaN,Mode,CramervMeasure,TschuprowtMeasure,CramervMeasure_Rank,TschuprowtMeasure_Rank,CramervFilter,CramervFilter_with,TschuprowtFilter,TschuprowtFilter_with
0,Categorical('Sex'),0.0,0.636364,0.533719,0.533719,5,5,0.260631,Parents/Children Aboard,0.260631,Parents/Children Aboard
1,Ordinal('Pclass'),0.0,0.548822,0.300144,0.300144,4,4,0.392245,Fare,0.392245,Fare
3,Quantitative('Fare'),0.0,0.831650,0.295325,0.295325,3,3,0.218058,Siblings/Spouses Aboard,0.183364,Siblings/Spouses Aboard
4,Quantitative('Siblings/Spouses Aboard'),0.0,0.680135,0.139722,0.117492,2,0,0.000000,itself,0.000000,itself
2,Quantitative('Age'),0.0,0.925926,0.139166,0.139166,1,2,0.466632,Parents/Children Aboard,0.466632,Parents/Children Aboard
5,Quantitative('Parents/Children Aboard'),0.0,0.737374,0.136439,0.136439,0,1,0.482783,Siblings/Spouses Aboard,0.405970,Siblings/Spouses Aboard


In [32]:
format_ranked_features(features, "Target")[1]

,feature,CramerV,CramerV_with,TschuprowT,TschuprowT_with
0,Categorical('Sex'),0.260631,Parents/Children Aboard,0.260631,Parents/Children Aboard
1,Ordinal('Pclass'),0.392245,Fare,0.392245,Fare
2,Quantitative('Age'),0.466632,Parents/Children Aboard,0.466632,Parents/Children Aboard
3,Quantitative('Fare'),0.218058,Siblings/Spouses Aboard,0.183364,Siblings/Spouses Aboard
4,Quantitative('Siblings/Spouses Aboard'),0.000000,itself,0.000000,itself
5,Quantitative('Parents/Children Aboard'),0.482783,Siblings/Spouses Aboard,0.405970,Siblings/Spouses Aboard


In [39]:
import pandas as pd

pd.DataFrame(filters)
# pd.DataFrame(measures)

,feature,TschuprowT,TschuprowT_with
0,Quantitative('Siblings/Spouses Aboard'),0.000000,itself
1,Quantitative('Parents/Children Aboard'),0.405970,Siblings/Spouses Aboard
2,Quantitative('Age'),0.466632,Parents/Children Aboard
3,Quantitative('Fare'),0.183364,Siblings/Spouses Aboard


In [40]:
pd.DataFrame(measures)

,feature,NaN,Mode,TschuprowT,TschuprowT_with
0,Quantitative('Siblings/Spouses Aboard'),0.0,0.680135,0.117492,target
1,Quantitative('Parents/Children Aboard'),0.0,0.737374,0.136439,target
2,Quantitative('Age'),0.0,0.925926,0.139166,target
3,Quantitative('Fare'),0.0,0.831650,0.295325,target


In [33]:
import pandas as pd

pd.DataFrame([format_measure(measure) for measure in measures])

,NaN,Mode,TschuprowT
0,0.0,0.636364,0.533719
1,0.0,0.548822,0.300144
2,0.0,0.925926,0.139166
3,0.0,0.831650,0.295325
4,0.0,0.680135,0.117492
5,0.0,0.737374,0.136439


In [ ]:

def _print_associations(self, association: DataFrame, message: str = "") -> None:
    """EDA of fitted associations

    Parameters
    ----------
    association : DataFrame
        _description_
    pretty_print : bool, optional
        _description_, by default False
    """
    # checking for verbose
    if self.verbose:
        print(f"\n - [{self.__name__}] Association between X and y{message}")

        # printing raw DataFrame
        if not self.pretty_print:
            print(association)

        # adding colors and displaying DataFrame as html
        else:
            # finding columns with indicators to colorize
            subset = [
                column
                for column in association.columns
                # checking for an association indicator
                if any(indic in column for indic in ["pct_", "_measure", "_filter"])
            ]

            # adding coolwarm color gradient
            nicer_association = association.style.background_gradient(
                cmap="coolwarm", subset=subset
            )
            # printing inline notebook
            nicer_association = nicer_association.set_table_attributes(
"style='display:inline'")

            # lower precision
            nicer_association = nicer_association.format(precision=4)

            # displaying html of colored DataFrame
            display_html(nicer_association._repr_html_(), raw=True)  # pylint: disable=W0212



In [33]:
best_features

[Quantitative('Siblings/Spouses Aboard'),
 Quantitative('Parents/Children Aboard'),
 Quantitative('Age'),
 Quantitative('Fare')]

In [34]:
best_features[1].measures

{'NaN': {'value': 0.0,
  'threshold': 1.0,
  'valid': True,
  'info': {'higher_is_better': False,
   'correlation_with': 'itself',
   'is_default': True}},
 'Mode': {'value': 0.7373737373737373,
  'threshold': 1.0,
  'valid': True,
  'info': {'higher_is_better': False,
   'correlation_with': 'itself',
   'is_default': True}},
 'TschuprowT': {'value': 0.13643860143826386,
  'threshold': 0.0,
  'valid': True,
  'info': {'higher_is_better': True,
   'correlation_with': 'target',
   'is_default': False}}}

In [35]:
features[2].filters

{'TschuprowT': {'value': 0.46663224633460926,
  'threshold': 1.0,
  'valid': True,
  'info': {'higher_is_better': False,
   'correlation_with': 'Parents/Children Aboard'}}}

In [55]:
features[1].filters

{'TschuprowT': {'value': 0.0,
  'threshold': 1.0,
  'valid': True,
  'info': {'higher_is_better': False, 'correlation_with': 'itself'}}}

* Feature ``Sex`` is the most associated with the target ``Survived``:
    - Tschuprow's T value is ``tschuprowt_measure=0.5337``
    - It has 0 % of NaNs (``pct_nan=0.0``) 
    - Its mode, ``0``, represents 64 % of observed data (``pct_nan=0.6364``)

* Here, no feature where filtered out for there inter-feature association or over-represented values (no thresholds were set)

## What's next?

* Thanks to **Carvers** all of your features are now optimally processed for your classification task!
* As a final step towards your model, **Selectors** can prove to be handy tools to operate target optimal Data Pre-Selection, so make sure to check out [Selectors Examples](https://autocarver.readthedocs.io/en/latest/selectors_examples.html)!

## Well done!

Your commitment to achieving optimal results in binary classification tasks shines through in your meticulous use of **AutoCarver**'s ``BinaryCarver`` for data preprocessing. By fine-tuning and optimizing your dataset, you have set the stage for robust and accurate machine learning models.

The ``BinaryCarver`` has proven to be a valuable ally in your pursuit of excellence, carving out a path toward enhanced feature representation and model interpretability. Your dedication to refining the data preprocessing steps reflects a commitment to extracting the maximum value from your datasets.

We extend our sincere appreciation for choosing **AutoCarver** as your companion in the data preprocessing journey. Your use of **AutoCarver** demonstrates a dedication to leveraging cutting-edge tools for achieving excellence in binary classification tasks.

As you transition to the modeling phase, may the carefully crafted features and preprocessing steps contribute to the success of your predictive models. We're excited to see the impact of your work and are grateful for the opportunity to be part of your data science endeavors.

Thank you for trusting **AutoCarver**, and we wish you continued success in your data-driven ventures.